In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import numpy as np
import pandas as pd
import torch.functional as F
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn import metrics
from focal_loss.focal_loss import FocalLoss
from imblearn.over_sampling import BorderlineSMOTE
from imblearn.over_sampling import SMOTE
from sklearn.preprocessing import LabelEncoder

In [2]:
data = pd.read_csv('Aggregated User Data.csv')
data.head()

,UUID,timestamp,raw_acc:magnitude_stats:mean,raw_acc:magnitude_stats:std,raw_acc:magnitude_stats:moment3,raw_acc:magnitude_stats:moment4,raw_acc:magnitude_stats:percentile25,raw_acc:magnitude_stats:percentile50,raw_acc:magnitude_stats:percentile75,raw_acc:magnitude_stats:value_entropy,...,label:STAIRS_-_GOING_DOWN,label:ELEVATOR,label:OR_standing,label:AT_SCHOOL,label:PHONE_IN_HAND,label:PHONE_IN_BAG,label:PHONE_ON_TABLE,label:WITH_CO-WORKERS,label:WITH_FRIENDS,label_source
0,00EABED2-271D-49D8-B599-1D4A09240601,1444079161,0.996815,0.003529,-0.002786,0.006496,0.995203,0.996825,0.998502,1.748756,...,NaN,NaN,0.0,NaN,NaN,NaN,1.0,1.0,NaN,2
1,00EABED2-271D-49D8-B599-1D4A09240601,1444079221,0.996864,0.004172,-0.003110,0.007050,0.994957,0.996981,0.998766,1.935573,...,NaN,NaN,0.0,NaN,NaN,NaN,1.0,1.0,NaN,2
2,00EABED2-271D-49D8-B599-1D4A09240601,1444079281,0.996825,0.003667,0.003094,0.006076,0.994797,0.996614,0.998704,2.031780,...,NaN,NaN,0.0,NaN,NaN,NaN,1.0,1.0,NaN,2
3,00EABED2-271D-49D8-B599-1D4A09240601,1444079341,0.996874,0.003541,0.000626,0.006059,0.995050,0.996907,0.998690,1.865318,...,NaN,NaN,0.0,NaN,NaN,NaN,1.0,1.0,NaN,2
4,00EABED2-271D-49D8-B599-1D4A09240601,1444079431,0.997371,0.037653,0.043389,0.102332,0.995548,0.996860,0.998205,0.460806,...,NaN,NaN,0.0,NaN,NaN,NaN,1.0,1.0,NaN,2


In [3]:
data = data[(data['UUID'] == '0BFC35E2-4817-4865-BFA7-764742302A2D') | (data['UUID'] == '0A986513-7828-4D53-AA1F-E02D6DF9561B') | (data['UUID'] == '00EABED2-271D-49D8-B599-1D4A09240601')] 
data.drop(columns = ['timestamp'], inplace = True)

# Creating a new dataframe with only sitting, walking, and sleeping data

In [4]:
only_walking = data[(data['label:FIX_walking'] == 1) & (data['label:SITTING'] == 0) & (data['label:SLEEPING'] == 0)]
only_walking = only_walking.iloc[:,1:27]
only_walking['label'] = "WALKING"

only_sitting = data[(data['label:FIX_walking'] == 0) & (data['label:SITTING'] == 1) & (data['label:SLEEPING'] == 0)]
only_sitting = only_sitting.iloc[:,1:27]
only_sitting['label'] = "SITTING"

only_sleeping = data[(data['label:FIX_walking'] == 0) & (data['label:SITTING'] == 0) & (data['label:SLEEPING'] == 1)]
only_sleeping = only_sleeping.iloc[:,1:27]
only_sleeping['label'] = "SLEEPING"

df = pd.concat([only_walking, only_sitting, only_sleeping], axis = 0)
df.head()

,raw_acc:magnitude_stats:mean,raw_acc:magnitude_stats:std,raw_acc:magnitude_stats:moment3,raw_acc:magnitude_stats:moment4,raw_acc:magnitude_stats:percentile25,raw_acc:magnitude_stats:percentile50,raw_acc:magnitude_stats:percentile75,raw_acc:magnitude_stats:value_entropy,raw_acc:magnitude_stats:time_entropy,raw_acc:magnitude_spectrum:log_energy_band0,...,raw_acc:3d:mean_x,raw_acc:3d:mean_y,raw_acc:3d:mean_z,raw_acc:3d:std_x,raw_acc:3d:std_y,raw_acc:3d:std_z,raw_acc:3d:ro_xy,raw_acc:3d:ro_xz,raw_acc:3d:ro_yz,label
155,1.003052,0.139768,0.115192,0.273656,0.991920,0.996238,0.999979,1.328158,6.674634,5.027699,...,-0.051719,-0.168622,-0.866269,0.250667,0.343037,0.252212,-0.030830,0.143926,-0.530033,WALKING
156,1.005091,0.232927,0.190056,0.292741,0.826441,0.948630,1.176819,2.745002,6.658351,5.013784,...,0.255740,-0.433965,-0.801786,0.276395,0.190099,0.235251,0.105712,0.016350,0.027738,WALKING
157,0.988339,0.025706,-0.012189,0.044177,0.979990,0.987836,0.996713,1.783550,6.684273,5.043187,...,-0.028900,-0.594037,-0.772274,0.136729,0.065971,0.065611,-0.603787,0.630796,-0.799983,WALKING
158,1.005461,0.105208,0.128558,0.199233,0.992822,0.994770,0.997684,1.459081,6.679381,5.039396,...,0.152049,-0.164488,-0.791628,0.272913,0.297443,0.426869,-0.372997,0.652161,-0.524372,WALKING
159,1.001464,0.121184,0.129053,0.209801,0.942805,0.996160,1.052143,2.060366,6.677517,5.029112,...,0.149694,-0.014688,-0.913222,0.299310,0.237561,0.122455,0.601753,-0.072091,-0.003273,WALKING


# Interpolating acceleration columns with average values

In [5]:
def interpolation(df):
    col_to_avg = list(df.columns) #Start with keeping all the columns as columns to use an average interpolation on
    for k in range(len(list(df.columns))):
        if list(df.columns)[k].startswith(('discrete', 'label')): #Remove label and discrete columns from col_to_avg
            col_to_avg.remove(list(df.columns)[k])
    
    df_with_avg = df[col_to_avg].fillna(df[col_to_avg].mean()) #Interpolate nan columns for all continuous-valued columns with average
    
    col_to_zero = list(df.columns)
    for k in range(len(list(df.columns))):
        if not list(df.columns)[k].startswith(('discrete', 'label')): #Remove all columns except label and discrete
            col_to_zero.remove(list(df.columns)[k])
    
    df_with_zero = df[col_to_zero].fillna(0) #Interpolate nan values for label and discrete columns with 0
    
    return pd.concat([df_with_avg, df_with_zero], axis = 1)

In [6]:
df.iloc[:,:-1] = interpolation(df.iloc[:,:-1]) #Interpolate acceleration columns
df = df.values
df

array([[1.003052, 0.139768, 0.115192, ..., 0.143926, -0.530033,
        'WALKING'],
       [1.005091, 0.232927, 0.190056, ..., 0.01635, 0.027738, 'WALKING'],
       [0.988339, 0.025706, -0.012189, ..., 0.630796, -0.799983,
        'WALKING'],
       ...,
       [1.000489, 0.001597, 0.000608, ..., -0.131635, 0.020957,
        'SLEEPING'],
       [1.000168, 0.003089, -0.0016879999999999998, ...,
        -0.08845900000000001, -0.090014, 'SLEEPING'],
       [1.019187, 0.143322, 0.24915, ..., 0.19007, -0.548567, 'SLEEPING']],
      dtype=object)

In [7]:
X = df[:, :-1]
y = df[:, -1]

y = LabelEncoder().fit_transform(y)

# SMOTE Oversampling

In [8]:
oversample = SMOTE()
X, y = oversample.fit_resample(X, y)

In [9]:
X.shape ### all classes are now the same size as the largest class

(10077, 26)

# Train/Test Split

In [15]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)
X_train

array([[-0.22727497, -0.44184899, -0.43345033, ...,  0.33117   ,
        -0.46216253, -1.9486312 ],
       [-0.39458493, -0.55496555, -0.52550688, ..., -0.09525267,
        -0.00205976, -0.19837935],
       [-0.24579742, -0.44192086, -0.48242486, ..., -0.25428205,
        -0.40565662, -1.67123746],
       ...,
       [ 0.23397145,  1.02177119,  0.69136307, ...,  0.53303707,
        -0.52500426, -0.16328479],
       [-0.31032384, -0.5555341 , -0.52029387, ..., -0.01696678,
         0.11783154, -0.00445485],
       [-0.29145215, -0.55623413, -0.52639103, ..., -0.03120995,
         0.24133314,  0.12355845]])

# Defining the model

In [17]:
def classifier_block(input_dim, output_dim):
    return nn.Sequential(
        nn.Linear(input_dim, output_dim),
        nn.Dropout(0.1),
        nn.LeakyReLU(0.05)
    )

class Classifier(nn.Module):
    def __init__(self, feature_dim = 26):
        super(Classifier, self).__init__()
        self.network = nn.Sequential(
            classifier_block(feature_dim, 20),
            classifier_block(20, 15),
            classifier_block(15, 10),
            classifier_block(10, 5),
            nn.Linear(5, 3)
        )
    def forward(self, x):
        #softmax = nn.Softmax(dim = 1)
        return self.network(x)

# Hyperparameters

In [21]:
model = Classifier()
lr = 0.01
n_epochs = 2000
batch_size = 100

### 2000 epochs 75 batch size 0.001 learning rate
#criterion = FocalLoss(alpha = 0.2, gamma = 5)

### Class 0 --> Sitting, Class 1 --> Sleeping, Class 2 --> Walking 
# weights = torch.tensor([1.0, 2.07, 13], dtype=torch.float32)
# weights = weights / weights.sum()
# normedWeights = torch.FloatTensor(weights)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr = lr)

train_features = torch.tensor(X_train)
train_labels = torch.tensor(y_train)
test_features = torch.tensor(X_test)
test_labels = torch.tensor(y_test)

train_data = torch.utils.data.TensorDataset(train_features, train_labels)
test_data = torch.utils.data.TensorDataset(test_features, test_labels)

train_loader = torch.utils.data.DataLoader(train_data, batch_size = batch_size, shuffle = True)
test_loader = torch.utils.data.DataLoader(test_data, batch_size = len(test_labels), shuffle = True)

# Training Loop

In [25]:
for epoch in range(n_epochs):
    total_loss = 0
    for batch in train_loader:
        features, labels = batch
        #_, encoded_labels = torch.max(labels, dim = 1) 
        
        optimizer.zero_grad()
        preds = model(features.float())
        
        loss = criterion(preds, labels.long()) 
        loss.backward()
        #torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm = 1.0)
        optimizer.step()
        total_loss += loss.item()
        
    print(f'Epoch {epoch + 1}, Loss: {total_loss}, Final Batch Loss: {loss.item()}')
#     softmax = nn.Softmax(dim = 1)
#     _, preds = torch.max(softmax(model(features.float())), dim = 1)
#     _, answers = torch.max(labels, dim = 1)
#     print(metrics.confusion_matrix(answers, preds))

Epoch 1, Loss: 60.54696089029312, Final Batch Loss: 0.5540499687194824
Epoch 2, Loss: 43.108387380838394, Final Batch Loss: 0.5488764047622681
Epoch 3, Loss: 38.73338142037392, Final Batch Loss: 0.441954642534256
Epoch 4, Loss: 37.28333841264248, Final Batch Loss: 0.5082456469535828
Epoch 5, Loss: 36.897125124931335, Final Batch Loss: 0.3232424557209015
Epoch 6, Loss: 35.080127477645874, Final Batch Loss: 0.2926487922668457
Epoch 7, Loss: 35.9518107175827, Final Batch Loss: 0.46968376636505127
Epoch 8, Loss: 34.58169123530388, Final Batch Loss: 0.6320382952690125
Epoch 9, Loss: 33.93429654836655, Final Batch Loss: 0.4184039235115051
Epoch 10, Loss: 32.28021363914013, Final Batch Loss: 0.4608815908432007
Epoch 11, Loss: 32.06535589694977, Final Batch Loss: 0.29122617840766907
Epoch 12, Loss: 32.255155459046364, Final Batch Loss: 0.42420393228530884
Epoch 13, Loss: 30.938568949699402, Final Batch Loss: 0.3620684742927551
Epoch 14, Loss: 31.646286338567734, Final Batch Loss: 0.35437715053

Epoch 114, Loss: 25.405117705464363, Final Batch Loss: 0.3114803433418274
Epoch 115, Loss: 24.893176078796387, Final Batch Loss: 0.2409253865480423
Epoch 116, Loss: 25.853358179330826, Final Batch Loss: 0.20868124067783356
Epoch 117, Loss: 25.988373532891273, Final Batch Loss: 0.3410746157169342
Epoch 118, Loss: 25.768147498369217, Final Batch Loss: 0.1870429366827011
Epoch 119, Loss: 25.94265589118004, Final Batch Loss: 0.4633902609348297
Epoch 120, Loss: 25.539234325289726, Final Batch Loss: 0.3414778411388397
Epoch 121, Loss: 25.401520743966103, Final Batch Loss: 0.34554624557495117
Epoch 122, Loss: 26.470883190631866, Final Batch Loss: 0.3533461093902588
Epoch 123, Loss: 25.23111516237259, Final Batch Loss: 0.22195996344089508
Epoch 124, Loss: 26.103471517562866, Final Batch Loss: 0.4700010120868683
Epoch 125, Loss: 24.887366250157356, Final Batch Loss: 0.2680172622203827
Epoch 126, Loss: 25.557771041989326, Final Batch Loss: 0.3000880181789398
Epoch 127, Loss: 25.527764320373535, 

Epoch 225, Loss: 23.98104737699032, Final Batch Loss: 0.3863951861858368
Epoch 226, Loss: 24.631139636039734, Final Batch Loss: 0.25638681650161743
Epoch 227, Loss: 24.140142112970352, Final Batch Loss: 0.27214962244033813
Epoch 228, Loss: 24.80043312907219, Final Batch Loss: 0.2695356011390686
Epoch 229, Loss: 24.183014899492264, Final Batch Loss: 0.3950718939304352
Epoch 230, Loss: 23.45309306681156, Final Batch Loss: 0.4029165506362915
Epoch 231, Loss: 24.826116755604744, Final Batch Loss: 0.3262425363063812
Epoch 232, Loss: 25.17193493247032, Final Batch Loss: 0.33509561419487
Epoch 233, Loss: 24.265648737549782, Final Batch Loss: 0.22038179636001587
Epoch 234, Loss: 24.344973981380463, Final Batch Loss: 0.34271669387817383
Epoch 235, Loss: 24.473578721284866, Final Batch Loss: 0.23548763990402222
Epoch 236, Loss: 24.45540940761566, Final Batch Loss: 0.3058697581291199
Epoch 237, Loss: 24.62779338657856, Final Batch Loss: 0.2618471086025238
Epoch 238, Loss: 24.63036400079727, Final

Epoch 336, Loss: 24.041999131441116, Final Batch Loss: 0.2119811624288559
Epoch 337, Loss: 22.822846308350563, Final Batch Loss: 0.13889101147651672
Epoch 338, Loss: 23.917911246418953, Final Batch Loss: 0.3230341374874115
Epoch 339, Loss: 23.37345589697361, Final Batch Loss: 0.17729805409908295
Epoch 340, Loss: 22.87772771716118, Final Batch Loss: 0.3537542521953583
Epoch 341, Loss: 23.560461848974228, Final Batch Loss: 0.4110853672027588
Epoch 342, Loss: 23.578437611460686, Final Batch Loss: 0.21762429177761078
Epoch 343, Loss: 23.150824919342995, Final Batch Loss: 0.252280056476593
Epoch 344, Loss: 23.973192989826202, Final Batch Loss: 0.43082138895988464
Epoch 345, Loss: 23.370668947696686, Final Batch Loss: 0.29178741574287415
Epoch 346, Loss: 23.683416426181793, Final Batch Loss: 0.39967289566993713
Epoch 347, Loss: 22.943858236074448, Final Batch Loss: 0.26951223611831665
Epoch 348, Loss: 24.710301235318184, Final Batch Loss: 0.3988376557826996
Epoch 349, Loss: 23.57052971422672

Epoch 447, Loss: 22.84789052605629, Final Batch Loss: 0.2956230044364929
Epoch 448, Loss: 21.85763119161129, Final Batch Loss: 0.3207380175590515
Epoch 449, Loss: 23.118713825941086, Final Batch Loss: 0.38799425959587097
Epoch 450, Loss: 22.444412045180798, Final Batch Loss: 0.15922218561172485
Epoch 451, Loss: 22.002069517970085, Final Batch Loss: 0.4181927442550659
Epoch 452, Loss: 23.352637752890587, Final Batch Loss: 0.39000409841537476
Epoch 453, Loss: 23.60058769583702, Final Batch Loss: 0.2736018896102905
Epoch 454, Loss: 23.09485137462616, Final Batch Loss: 0.2904118001461029
Epoch 455, Loss: 22.744267478585243, Final Batch Loss: 0.21563401818275452
Epoch 456, Loss: 22.402082189917564, Final Batch Loss: 0.3326750099658966
Epoch 457, Loss: 21.907884046435356, Final Batch Loss: 0.23963069915771484
Epoch 458, Loss: 22.346005454659462, Final Batch Loss: 0.4346686005592346
Epoch 459, Loss: 21.80356276035309, Final Batch Loss: 0.4986408054828644
Epoch 460, Loss: 23.16727042198181, Fi

...

Epoch 1549, Loss: 19.87595873326063, Final Batch Loss: 0.20078496634960175
Epoch 1550, Loss: 19.50442361831665, Final Batch Loss: 0.2545871436595917
Epoch 1551, Loss: 19.12789824604988, Final Batch Loss: 0.4827306270599365
Epoch 1552, Loss: 19.559867538511753, Final Batch Loss: 0.35648372769355774
Epoch 1553, Loss: 18.25515802204609, Final Batch Loss: 0.16453510522842407
Epoch 1554, Loss: 19.518114037811756, Final Batch Loss: 0.23022891581058502
Epoch 1555, Loss: 19.020554415881634, Final Batch Loss: 0.1777753233909607
Epoch 1556, Loss: 19.27568083256483, Final Batch Loss: 0.2454739660024643
Epoch 1557, Loss: 18.233357645571232, Final Batch Loss: 0.11188466101884842
Epoch 1558, Loss: 19.55773176252842, Final Batch Loss: 0.2832060158252716
Epoch 1559, Loss: 19.146487593650818, Final Batch Loss: 0.16459733247756958
Epoch 1560, Loss: 19.379728130996227, Final Batch Loss: 0.32071319222450256
Epoch 1561, Loss: 19.194799832999706, Final Batch Loss: 0.25946080684661865
Epoch 1562, Loss: 18.19

Epoch 1658, Loss: 20.27873957157135, Final Batch Loss: 0.4331849217414856
Epoch 1659, Loss: 18.591710552573204, Final Batch Loss: 0.14901220798492432
Epoch 1660, Loss: 19.760580241680145, Final Batch Loss: 0.2985776364803314
Epoch 1661, Loss: 19.17980781197548, Final Batch Loss: 0.1818990856409073
Epoch 1662, Loss: 19.094402380287647, Final Batch Loss: 0.32933247089385986
Epoch 1663, Loss: 19.215761229395866, Final Batch Loss: 0.22711390256881714
Epoch 1664, Loss: 19.1330082193017, Final Batch Loss: 0.21905003488063812
Epoch 1665, Loss: 19.84157457947731, Final Batch Loss: 0.27225324511528015
Epoch 1666, Loss: 19.958756133913994, Final Batch Loss: 0.21095412969589233
Epoch 1667, Loss: 19.48213267326355, Final Batch Loss: 0.21596795320510864
Epoch 1668, Loss: 18.79440873861313, Final Batch Loss: 0.1697758287191391
Epoch 1669, Loss: 19.056128457188606, Final Batch Loss: 0.17364783585071564
Epoch 1670, Loss: 19.073963180184364, Final Batch Loss: 0.12084658443927765
Epoch 1671, Loss: 18.78

Epoch 1767, Loss: 19.540491320192814, Final Batch Loss: 0.19069504737854004
Epoch 1768, Loss: 19.65934333205223, Final Batch Loss: 0.1981544941663742
Epoch 1769, Loss: 18.56972686946392, Final Batch Loss: 0.46191251277923584
Epoch 1770, Loss: 19.24350008368492, Final Batch Loss: 0.3798990547657013
Epoch 1771, Loss: 18.185836635529995, Final Batch Loss: 0.3689616918563843
Epoch 1772, Loss: 19.919222086668015, Final Batch Loss: 0.32189229130744934
Epoch 1773, Loss: 18.889980494976044, Final Batch Loss: 0.22905677556991577
Epoch 1774, Loss: 19.075097374618053, Final Batch Loss: 0.1835830807685852
Epoch 1775, Loss: 18.697104655206203, Final Batch Loss: 0.24630919098854065
Epoch 1776, Loss: 18.637099541723728, Final Batch Loss: 0.24377718567848206
Epoch 1777, Loss: 18.664806693792343, Final Batch Loss: 0.2408285290002823
Epoch 1778, Loss: 18.849792502820492, Final Batch Loss: 0.2890242338180542
Epoch 1779, Loss: 19.442463107407093, Final Batch Loss: 0.15331712365150452
Epoch 1780, Loss: 18.

Epoch 1876, Loss: 18.868761241436005, Final Batch Loss: 0.18944445252418518
Epoch 1877, Loss: 17.633999317884445, Final Batch Loss: 0.176827073097229
Epoch 1878, Loss: 19.281953640282154, Final Batch Loss: 0.3125261962413788
Epoch 1879, Loss: 19.359795711934566, Final Batch Loss: 0.24836526811122894
Epoch 1880, Loss: 19.226949878036976, Final Batch Loss: 0.1975635141134262
Epoch 1881, Loss: 18.612114779651165, Final Batch Loss: 0.13879641890525818
Epoch 1882, Loss: 18.659451514482498, Final Batch Loss: 0.14151151478290558
Epoch 1883, Loss: 18.233129806816578, Final Batch Loss: 0.08407442271709442
Epoch 1884, Loss: 20.13044425100088, Final Batch Loss: 0.2608189582824707
Epoch 1885, Loss: 20.62657506018877, Final Batch Loss: 0.2731369733810425
Epoch 1886, Loss: 18.9618588835001, Final Batch Loss: 0.27536317706108093
Epoch 1887, Loss: 19.723490364849567, Final Batch Loss: 0.340870201587677
Epoch 1888, Loss: 19.75553473830223, Final Batch Loss: 0.2815817594528198
Epoch 1889, Loss: 18.79876

Epoch 1985, Loss: 18.99714954942465, Final Batch Loss: 0.2750300168991089
Epoch 1986, Loss: 18.94077469408512, Final Batch Loss: 0.14482374489307404
Epoch 1987, Loss: 19.02041868120432, Final Batch Loss: 0.2280864268541336
Epoch 1988, Loss: 19.281508058309555, Final Batch Loss: 0.2733904719352722
Epoch 1989, Loss: 19.004660055041313, Final Batch Loss: 0.23495398461818695
Epoch 1990, Loss: 18.840773791074753, Final Batch Loss: 0.21072925627231598
Epoch 1991, Loss: 19.341188192367554, Final Batch Loss: 0.17826850712299347
Epoch 1992, Loss: 19.230675615370274, Final Batch Loss: 0.5380011200904846
Epoch 1993, Loss: 18.16287077218294, Final Batch Loss: 0.18941882252693176
Epoch 1994, Loss: 18.602202221751213, Final Batch Loss: 0.15581347048282623
Epoch 1995, Loss: 18.9716774597764, Final Batch Loss: 0.36828356981277466
Epoch 1996, Loss: 18.959380224347115, Final Batch Loss: 0.19645121693611145
Epoch 1997, Loss: 18.510531775653362, Final Batch Loss: 0.17064909636974335
Epoch 1998, Loss: 19.0

# Evaluation on test data

In [29]:
softmax = nn.Softmax(dim = 1)
for batch in test_loader:
    features, labels = batch
    _, preds = torch.max(softmax(model(features.float())), dim = 1)
    #_, answers = torch.max(labels, dim = 1)
    print(metrics.confusion_matrix(labels, preds))
    print(metrics.classification_report(labels, preds, digits = 3))

[[548  48  59]
 [ 27 681   5]
 [ 44   9 595]]
              precision    recall  f1-score   support

           0      0.885     0.837     0.860       655
           1      0.923     0.955     0.939       713
           2      0.903     0.918     0.910       648

    accuracy                          0.905      2016
   macro avg      0.904     0.903     0.903      2016
weighted avg      0.904     0.905     0.904      2016

